<a href="https://colab.research.google.com/github/itsmihhhhhh/Luno-CryptoTradingBot/blob/master/XRPBTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install required packages

In [ ]:
!pip install luno_python
!pip install mpl_finance
!pip install pyti
!pip install numpy
!pip install tulipy
import tulipy as ti

#Connect with Luno account

In [ ]:
from luno_python.client import Client
c = Client(api_key_id='dfqpu5an2kh7q', api_key_secret='UWjQZBJxoNEOKL1id59VaFBSBORvuAdeR2ZHA6_vdso')
try:
  ticker = c.get_ticker(pair='XRPXBT')
  print(ticker)
except Exception as e:
  print(e)

ticker['last_trade']

In [ ]:
res = c.get_order_book(pair='XRPXBT')


In [ ]:
balance = c.get_balances(assets=['XRP','XBT'])
balXBT = balance['balance'][0]['balance']
balXRP = balance['balance'][1]['balance']
balXRP

In [ ]:
enter_trade = True

In [ ]:
import sched, time
import numpy as np


#Initiate our scheduler so we can keep checking every minute for new price changes
s = sched.scheduler(time.time, time.sleep)

# This function is run every 5 minutes

def run(sc): 

    #Get prices from Luno for LTC
    res = c.get_order_book(pair='XRPXBT')
    closePrices = c.list_trades(pair='XRPXBT')
    pricedata = []
    for x in range(0,len(closePrices['trades'])):
        pricedata.append(float(closePrices['trades'][x]['price']))
    DATA = np.array(pricedata)

    askdata = []
    for x in range(0,len(res['asks'])):
        askdata.append(float(res['asks'][x]['price']))
    DATAA = np.array(askdata)

    biddata = []
    for x in range(0,len(res['bids'])):
        biddata.append(float(res['bids'][x]['price']))
    DATAB = np.array(biddata)

    #Calculate Indicators
    #iRSI = ti.rsi(DATA, period=14)
    BB = ti.bbands(DATA, period=20, stddev=2)
    BB1 = ti.bbands(DATA, period=20, stddev=1)
    MA = ti.sma(DATA, period=5)
    MAA = ti.sma(DATAA, period=3)
    MAB = ti.sma(DATAB, period=5)

    #TRADING LOGIC using Moving average line
    '''
    if (MAB[0] < pricedata[0] < MA[0]):
        print("Buying, price is low!")
        enter("B")

    if (MA[0] < pricedata[0] < MAA[0]):
        print("Selling, price is high!")
        enter("S")
    '''

    #TRADING LOGIC with Bollinger Band            
    if (BB[1][0] > pricedata[0] > BB[0][0] and float(balXRP)<90):
        print("Buying, price is low!")
        try:
          '''
          if (price == float(res['asks'][0]['price'])):
            c.post_market_order(pair='XRPXBT', type='BUY', counter_volume=0.0023)
            print("Buy Successfully at price.", price)
            enter_trade = False
          else:
          '''
          c.post_limit_order(pair='XRPXBT', type='BID', volume='30', price = ticker['last_trade'])
          print("Post limit order. Buy Successfully at price: ", ticker['last_trade'])
          enter_trade = True
        except Exception as e:
          print(e)
          enter_trade = False

    if (BB[1][0] < pricedata[0] < BB[2][0]):
        print("Selling, price is high!")
        try:
          '''
          if (price == float(res['bids'][0]['price'])):
            c.post_market_order(pair='XRPXBT', type='SELL', base_volume='80')
            print("Sell Successfully at price.", price)
            enter_trade = False
          else:
            '''
          c.post_limit_order(pair='XRPXBT', type='ASK', volume='30', price = ticker['last_trade'])
          print("Post limit order. Sell Successfully at price.", ticker['last_trade'])
          enter_trade1 = False
        except Exception as e:
          print(e)
          enter_trade1 = True

    '''
    if (BB1[0][0] > current_price > BB[0][0]):
        print("Price might go down, sell to cut lost")
        #enter("S")

    if (BB1[2][0] < current_price < BB[2][0]):
        print("Price might go up, buy now")
        #enter("B")
    '''    

    print("--------------------------------------")
    print("Current price: ", pricedata[0])  
    
    print("MA bid(low): ", MAB[0])
    print("MA middle: ", MA[0])
    print("MA ask(high): ", MAA[0])
    print("BBand lower: ", BB[0][0])
    print("BBand middle: ", BB[1][0])
    print("BBand upper ", BB[2][0])
  
    print("                               Bot working, checking...")

    #call this method again every 5 minutes for new price changes
    s.enter(1000, 1, run, (sc,))
    
s.enter(1, 1, run, (s,))

s.run()